In [17]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [18]:
# Inputs
new_parcels = r".\new_remm_base_year_2019.gdb\parcels_2019"
old_parcels = r".\old_remm_base_year_2019.gdb\parcels_2019"

In [19]:
# output directory
if not os.path.exists('Outputs'): os.makedirs('Outputs') 
outputs = ['.\\Outputs', 'remm_base_year_2019.gdb']
gdb = os.path.join(outputs[0], outputs[1])
if not arcpy.Exists(gdb): arcpy.CreateFileGDB_management(outputs[0], outputs[1])

In [20]:
pts = arcpy.FeatureToPoint_management(old_parcels, os.path.join(gdb, 'new_pts'), point_location='inside')

In [21]:
target_features = pts
join_features = new_parcels
output_features = os.path.join(gdb, 'sj')

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# have to specify since both datasets contain these fields??
# fieldmappings.addInputField(join_features, 'max_dua')
# fieldmappings.addInputField(join_features, 'max_far')

# max dua
fieldindex = fieldmappings.findFieldMapIndex('max_dua')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# max far
fieldindex = fieldmappings.findFieldMapIndex('max_far')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Max'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "INTERSECT")

In [22]:
old_parcels_sdf = pd.DataFrame.spatial.from_featureclass(old_parcels)
sj_df = pd.DataFrame.spatial.from_featureclass(sj[0])[['parcel_id_REMM', 'max_dua', 'max_far']].copy()
sj_df.columns = ['parcel_id_REMM', 'max_dua_NEW', 'max_far_NEW']

In [23]:
merged = old_parcels_sdf.merge(sj_df, on='parcel_id_REMM', how='left')

merged['max_dua'] = merged['max_dua_NEW']
merged['max_far'] = merged['max_far_NEW']

del merged['max_dua_NEW']
del merged['max_far_NEW']

In [24]:
result = os.path.join(gdb, 'parcels_2019')
merged.spatial.to_featureclass(location=result, sanitize_columns=False)

'e:\\Projects\\Apply_Zoning_to_Old_Parcels\\Outputs\\remm_base_year_2019.gdb\\parcels_2019'